# Building a CKD Classifier for IgAN Patients

### Steps To Building Model
1. Load Data
2. Clean Data
3. Split Data
4. Build Model
5. Export weights

### Loading Data
To perform this step, the CKD dataset must be downloaded and moved to the working directory

The dataset can be found here ---> http://archive.ics.uci.edu/ml//datasets/Chronic_Kidney_Disease

In [64]:
import arff
import pandas as pd

data_path = 'Chronic_Kidney_Disease/chronic_kidney_disease.arff'

with open(data_path, 'r') as file:
    data = file.read()
    
# remove double commas and trailing commas
data = data.replace(',,', ',');
data = data.replace(',\n', '\n');
    

with open(data_path, 'w') as file:
    file.write(data)


In [65]:
data = arff.load(open(data_path, 'r'))

ckd_df = pd.DataFrame(data['data'])

ckd_df.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,48.0,80.0,1.020,1,0,None,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4,0,None,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2,3,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4,0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2,0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,ckd
